In [11]:
from openai import AssistantEventHandler
from typing_extensions import override
from openai import OpenAI

In [18]:
client = OpenAI(api_key="")

In [19]:

assistant = client.beta.assistants.create(
    name="Math Tutor 2",
    instructions="You are a personal math tutor. Write and run code to answer math questions.",
    tools=[{"type": "code_interpreter"}],
    model="gpt-4o",
)


In [20]:
my_assistants = client.beta.assistants.list(
    order="desc",
    limit="20",
)
print([f"assitant {a.id} and name: {a.name}" for a in my_assistants.data])

['assitant asst_XKgCX6FxxFmpVsRtLm2jfUrd and name: Math Tutor 2', 'assitant asst_sXqac6YoYlGTNdse3b8EUNpn and name: Math Tutor 2', 'assitant asst_QVDKNFQ9SSZpgK0DVk2YfmcH and name: None', 'assitant asst_HCgVGOAC9TzDnmwuOyxTFRAv and name: None', 'assitant asst_TCOYhPbHEO5hXLu5ehFjDAIM and name: None', 'assitant asst_01yHqpQoeNt3cHi3cLvyqMDb and name: None', 'assitant asst_AXlTFxeamCi0fn4MHW6l0AS9 and name: None', 'assitant asst_7Z1gx97pSDJibVka6ULwdNQy and name: None', 'assitant asst_M0RLsAHYUyxYidCqSCl9neUB and name: None', 'assitant asst_lvhw7EQnp3dyIO2Q7Q2L0Nna and name: None', 'assitant asst_gwQbAXP4T32KGUVZrIZpJe5z and name: None', 'assitant asst_AMJg7Z5nUY32bylBMG0hmmc1 and name: None', 'assitant asst_EdhLmGlUIBsznEg5M1cxvAtg and name: None', 'assitant asst_DhPdtA1YMwU2flBzShXgrg5V and name: None', 'assitant asst_4k2Gdu8E1KKChBnnSEbrRPU8 and name: None', 'assitant asst_NbzHABPpL9r6aP2h6y3K8rKb and name: None', 'assitant asst_9mIwq2VUqwvEZgQs6SClszSl and name: None', 'assitant asst

In [21]:

thread = client.beta.threads.create()

In [22]:
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="I need to solve the equation `3x + 11 = 14`. Can you help me?"
)


# First, we create a EventHandler class to define
# how we want to handle the events in the response stream.

class EventHandler(AssistantEventHandler):
    @override
    def on_text_created(self, text) -> None:
        print(f"\nassistant > ", end="", flush=True)

    @override
    def on_text_delta(self, delta, snapshot):
        print(delta.value, end="", flush=True)

    def on_tool_call_created(self, tool_call):
        print(f"\nassistant > {tool_call.type}\n", flush=True)

    def on_tool_call_delta(self, delta, snapshot):
        if delta.type == 'code_interpreter':
            if delta.code_interpreter.input:
                print(delta.code_interpreter.input, end="", flush=True)
            if delta.code_interpreter.outputs:
                print(f"\n\noutput >", flush=True)
                for output in delta.code_interpreter.outputs:
                    if output.type == "logs":
                        print(f"\n{output.logs}", flush=True)

# Then, we use the `stream` SDK helper
# with the `EventHandler` class to create the Run
# and stream the response.


with client.beta.threads.runs.stream(
    thread_id=thread.id,
    assistant_id=assistant.id,
    instructions="Please address the user as Jane Doe. The user has a premium account.",
    event_handler=EventHandler(),
) as stream:
    stream.until_done()




assistant > Of course, Jane Doe! Let's solve the equation \( 3x + 11 = 14 \) step by step.

1. Start with the given equation:
   \[
   3x + 11 = 14
   \]

2. Subtract 11 from both sides of the equation to isolate the term with \( x \):
   \[
   3x + 11 - 11 = 14 - 11
   \]
   Simplifying this, we get:
   \[
   3x = 3
   \]

3. Divide both sides by 3 to solve for \( x \):
   \[
   \frac{3x}{3} = \frac{3}{3}
   \]
   Simplifying this, we get:
   \[
   x = 1
   \]

So, the solution to the equation \( 3x + 11 = 14 \) is \( x = 1 \).

In [23]:
class StreamEventHandler(AssistantEventHandler):
    # Iterator over just the text deltas in the stream
    @override
    def on_text_created(self, text) -> None:
        # Callback that is fired when a text content block is created
        # print(f"\nassistant > ", end="", flush=True)
        pass

    @override
    def on_text_delta(self, delta, snapshot):
        """Callback that is fired whenever a text content delta is returned
        by the API.

        The first argument is just the delta as sent by the API and the second argument
        is the accumulated snapshot of the text. For example:

        on_text_delta(TextDelta(value="The"), Text(value="The")),
        on_text_delta(TextDelta(value=" solution"), Text(value="The solution")),
        on_text_delta(TextDelta(value=" to"), Text(value="The solution to")),
        on_text_delta(TextDelta(value=" the"), Text(value="The solution to the")),
        on_text_delta(TextDelta(value=" equation"), Text(value="The solution to the equivalent")),
        """
        # print(delta.value, end="", flush=True)
        pass

In [24]:
stream = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=assistant.id,
  stream=True
)

for event in stream:
  print(event)
  if event.event != "thread.message.delta":
    continue

  # for content_delta in event.data.delta.content or []:
  #   if content_delta.type == "text" and content_delta.text and content_delta.text.value:
  #       print(content_delta.text.value)
  print(event.data.delta.content[0].text.value)


ThreadRunCreated(data=Run(id='run_3lsHyyTQcIsaTZeph9mTI651', assistant_id='asst_XKgCX6FxxFmpVsRtLm2jfUrd', cancelled_at=None, completed_at=None, created_at=1721317688, expires_at=1721318288, failed_at=None, incomplete_details=None, instructions='You are a personal math tutor. Write and run code to answer math questions.', last_error=None, max_completion_tokens=None, max_prompt_tokens=None, metadata={}, model='gpt-4o', object='thread.run', parallel_tool_calls=True, required_action=None, response_format='auto', started_at=None, status='queued', thread_id='thread_2MkycVANM3eHGol2C19Imrm2', tool_choice='auto', tools=[CodeInterpreterTool(type='code_interpreter')], truncation_strategy=TruncationStrategy(type='auto', last_messages=None), usage=None, temperature=1.0, top_p=1.0, tool_resources={'code_interpreter': {'file_ids': []}}), event='thread.run.created')
ThreadRunQueued(data=Run(id='run_3lsHyyTQcIsaTZeph9mTI651', assistant_id='asst_XKgCX6FxxFmpVsRtLm2jfUrd', cancelled_at=None, completed_

In [25]:
with client.beta.threads.runs.stream(
    thread_id=thread.id,
    assistant_id=assistant.id,
) as stream:
    if stream.on_text_created:
        print('@@@@ start the big bang')
    if stream.on_text_delta:
        # for text in stream.text_deltas:
        # print(*stream.text_deltas, end="", flush=True)
        print('-------')
        print(stream.)
    if stream.on_end:
        print('####syaonara')

SyntaxError: invalid syntax (1711408863.py, line 11)

In [ ]:
assistant = client.beta.assistants.retrieve(
                "session.assistant_id")

NotFoundError: Error code: 404 - {'error': {'message': "No assistant found with id 'session.assistant_id'.", 'type': 'invalid_request_error', 'param': None, 'code': None}}